# Prédiction de l'impact de sanctions économiques

## Notebook 1 - Modèle "naïf" (pas de feature engineering)

In [1]:
# import required libraries

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 80)
sns.set_style("whitegrid")
%matplotlib inline

np.random.seed(0)

In [2]:
# import the data

base_df = pd.read_csv("input/sanctions.csv.gz", index_col=0)
print(base_df.shape)
# for col, typ in zip(base_df.columns, base_df.dtypes):
#     print(col, "=>", typ)
base_df.head(3)

(204, 38)


,Sender c,Sender 2c,Sender 3c,Targetd,Goal,US casee,US unilateral casef,Foreign policy goal categoryg,First yearh,Last yeari,Policy resultj,Sanctions contributionk,Success \nscorel,Companion policiesm,International cooperationn,International assistanceo,Cooperating international organizationp,International organization senderq,International organization sender & target membersr,Length (years)s,Prior relationst,Regime Type \n(DEMOC 1st year)u,Regime Type2 (Polity2 1st year)v,Regime Type3 \n(Scale 1st year)w,Political stability priorx,Political stability duringy,Cost to targetz,Cost to target (percent of GNP)aa,Cost to target per capitabb,Trade linkagecc,GNP \nratiodd,Health and stabilityee,Sanction typeff,Cost to sendergg,"GDP growth (percent, 5-year average)hh","Inflation (percent,\n 3-year average)ii",Target International Monetary Fund codejj,Country groupkk
Case no.b,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14-1,UNITED KINGDOM,NaN,NaN,GERMANY,Military victory,0,0,4,1914,1918,4,3,12,R,4,A,--,0,0,4,1,5,2,2,"0,0","0,0",843,"7,1","12,58",9,1,3,"F,X,M",4,"3,6","2,9",134,1
17-1,UNITED STATES,NaN,NaN,JAPAN,Shipping for Allies,1,1,5,1917,1918,2,2,4,--,1,--,--,0,0,1,2,5,1,2,"0,0","0,0",23,"0,8","0,44","20,5",13,3,X,2,"5,4","25,6",158,1
18-1,UNITED KINGDOM,NaN,NaN,RUSSIA,Destabilize Bolsheviks,0,0,2,1918,1920,1,2,2,"R,Q",4,--,--,0,0,2,1,1,-1,2,"0,1","0,0",446,"4,1","2,49","18,5",1,1,"F,X,M",3,n.a.,n.a.,922,2


In [3]:
# Drop information that is unknown at decision time

unknowns = [
    "Last yeari",
    "Length (years)s",
    "Political stability duringy",
    "Cost to targetz",
    "Cost to target (percent of GNP)aa",
    "Cost to target per capitabb",
    "Cost to sendergg",
    "Target International Monetary Fund codejj"
]

df = base_df.drop(unknowns, axis=1)

In [4]:
# Rename all valid columns

new_names = [
    'Sender1', 'Sender2', 'Sender3', 'Target', 'Goal', 'US case',
    'US unilateral case', 'Foreign policy goal category', 'First year',
    'Policy result', 'Sanctions contribution', 'Success',
    'Companion policies', 'International cooperation',
    'International assistance', 'Cooperating international organization',
    'International organization sender',
    'International organization sender & target members',
    'Prior relationst', 'Regime Type',
    'Regime Type2', 'Regime Type3',
    'Political stability prior', 'Trade linkage', 'GNP \nratio',
    'Health and stability', 'Sanction type',
    'GDP growth',
    'Inflation', 'Country group']

name_dict = dict(zip(df.columns, new_names))
name_dict

df = df.rename(columns=name_dict)

In [5]:
# Make "Success" either 1 or 0
df["score"] = (df['Success'] > 9).astype(int)

In [7]:
df.head()

,Sender1,Sender2,Sender3,Target,Goal,US case,US unilateral case,Foreign policy goal category,First year,Policy result,Sanctions contribution,Success,Companion policies,International cooperation,International assistance,Cooperating international organization,International organization sender,International organization sender & target members,Prior relationst,Regime Type,Regime Type2,Regime Type3,Political stability prior,Trade linkage,GNP \nratio,Health and stability,Sanction type,GDP growth,Inflation,Country group,score
Case no.b,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14-1,UNITED KINGDOM,NaN,NaN,GERMANY,Military victory,0,0,4,1914,4,3,12,R,4,A,--,0,0,1,5,2,2,"0,0",9,1,3,"F,X,M","3,6","2,9",1,1
17-1,UNITED STATES,NaN,NaN,JAPAN,Shipping for Allies,1,1,5,1917,2,2,4,--,1,--,--,0,0,2,5,1,2,"0,0","20,5",13,3,X,"5,4","25,6",1,0
18-1,UNITED KINGDOM,NaN,NaN,RUSSIA,Destabilize Bolsheviks,0,0,2,1918,1,2,2,"R,Q",4,--,--,0,0,1,1,-1,2,"0,1","18,5",1,1,"F,X,M",n.a.,n.a.,2,0
21-1,LEAGUE OF NATIONS,NaN,NaN,YUGOSLAVIA,Military disruption vs. Albania,0,0,3,1921,4,4,16,--,4,--,--,1,1,2,3,0,2,"0,4","26,5",37,2,--,"-1,5",n.a.,2,1
25-1,LEAGUE OF NATIONS,NaN,NaN,GREECE,Withdraw from Bulgaria,0,0,3,1925,4,4,16,--,4,--,--,1,1,2,0,-6,1,"0,9",36,56,2,--,"3,3","48,8",1,1


### Ici je pense qu'on peut reprendre la plupart du code présent dans le notebook "Guillaume"

...

...

...

## Entrainement

In [ ]:
rf_pipeline = {
    'rf' : make_pipeline(StandardScaler(), RandomForestClassifier(random_state = 42))
}

rf__hyperparameters = {
    'randomforestclassifier__n_estimators' : [100, 200],
    'randomforestclassifier__max_features' : ['auto', 'sqrt', 0.33],
    'randomforestclassifier__min_samples_leaf' : [1, 3, 5, 10]
}

model = GridSearchCV(rf_pipeline, rf__hyperparameters, cv= 10, n_jobs= -1)
#model.fit(x_train, y_train)